# generate inp files ans split

- 2 steps:
    - generate inp files from xyz files or other formats
    - split inp files for multi threads


In [82]:
# set project root
import os
import sys
import rootutils

rootutils.set_root(
    path=rootutils.find_root(search_from='.', indicator='.env'),
    project_root_env_var=True,  # set PROJECT_ROOT env var
    dotenv=True,  # load dotenv file
    cwd=True,  # change working directory
    pythonpath=False,  # add project root to PYTHONPATH
)

PROJECT_ROOT = os.environ["PROJECT_ROOT"]
os.chdir(PROJECT_ROOT)
PROJECT_ROOT

'/home/louzekun/orca_cal/qm7_ccsd_t_new'

Here is something you should modify according to your own task

In [83]:
xyz_dname = "xyz"
inp_dname = "inp"
out_dname = "out"
threads_dname = "threads"

CAL_CMD = "! HF DLPNO-CCSD(T) DEF2-QZVPP DEF2-QZVPP/C TightSCF NormalPNO UseSym"
# use a simpler method to calculate energy
# CAL_CMD = "! PBE DEF2-TZVP LooseSCF UseSym"

SPLIT_CNT = 3  # start N orca tasks

## generate inp files

- in this part, you don't have to change anything, just run the code
- and rescue the code if it goes wrong

In [84]:
import os
import re
import os.path as osp
from typing import Union, List, Dict

from tqdm import tqdm

os.chdir(PROJECT_ROOT)
os.getcwd()

'/home/louzekun/orca_cal/qm7_ccsd_t_new'

In [85]:
xyz_fnames = [i for i in os.listdir(xyz_dname) if i.endswith(".xyz")]
print(f"{len(xyz_fnames)=}")


len(xyz_fnames)=7165


In [86]:
def parse_xyz_coords(coords_str:List[str]) -> List[List[Union[str, List[float]]]]:
    """
    input: ["O  -3.56626  1.77639  0.00000", ...]
    output: [[atom, [x, y, z]], ...]
    """
    coords_list = []
    regex = re.compile(r"([A-Z][a-z]*)\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)")
    for line in coords_str:
        m = regex.match(line)
        if m:
            coords_list.append([m.group(1), [float(m.group(2)), float(m.group(3)), float(m.group(4))]])
    return coords_list


def convert_xyz2inp(
    xyz_fpath:str,
    inp_fpath:str,
    cal_cmd:str,  # like "! HF/6-31G(d) OPT"
    nproc:Union[int, str],
    maxcore:int = 2000,
    charge:int = 0,
    multiplicity:int = 1,
    test=False,  # don't write to inp file
):
    """Convert xyz file to inp file

    Supported % commands:
    - f'%PAL NPROC {nproc} END'
        - nproc = max(nelec, nproc)
    - f'%MAXCORE {maxcore}'

    `.xyz` file format example: see https://en.wikipedia.org/wiki/XYZ_file_format
    """

    # some preparations
    ELEC = {"H":1, "He":2, "Li":3, "Be":4, "B":5, "C":6, "N":7, "O":8, "F":9, "Ne":10,
            "Na":11, "Mg":12, "Al":13, "Si":14, "P":15, "S":16, "Cl":17, "Ar":18, "K":19, "Ca":20}
    # B2A = 0.529177208  # Bohr to Angstrom

    # parse xyz file
    with open(xyz_fpath, "r") as f:
        xyz_coords_str = f.readlines()[2:]  # skip natom and comment
    xyz_coords_str = [line.strip('\n') for line in xyz_coords_str]
    coords = parse_xyz_coords(xyz_coords_str)
    nelec = sum([ELEC[atom] for atom, _ in coords]) - charge  # charge +1 -> e- one fewer
    if nproc is None:
        nproc = nelec
    else:
        nproc = min(nelec, nproc)

    # prepare inp text and write
    # inp_coords_str = [f"{atom} {x} {y} {z}" for atom, [x, y, z] in coords]
    inp_coords_str = xyz_coords_str
    inp_lines = [
        cal_cmd,
        f"%PAL NPROC {nproc} END",
        f"%MAXCORE {maxcore}",
        f"* xyz {charge} {multiplicity}",
    ] + inp_coords_str + ["*", ]
    inp_text = "\n".join(inp_lines)
    if test:
        return inp_text
    else:
        with open(inp_fpath, "w") as f:
            f.write(inp_text)



In [87]:
print(convert_xyz2inp(
    osp.join(xyz_dname, xyz_fnames[0]), None,
    cal_cmd=CAL_CMD, nproc=None, test=True))

! HF DLPNO-CCSD(T) DEF2-QZVPP DEF2-QZVPP/C TightSCF NormalPNO UseSym
%PAL NPROC 10 END
%MAXCORE 2000
* xyz 0 1
C       0.99826000     -0.00246000     -0.00436000
H       2.09021000     -0.00243000      0.00414000
H       0.63379000      1.02685999      0.00414000
H       0.62704001     -0.52772999      0.87811003
H       0.64136001     -0.50746999     -0.90539998
*


In [88]:
for xyz_fname in tqdm(xyz_fnames, ncols=80):
    task_name = xyz_fname.replace(".xyz", "")
    inp_fname = task_name + ".inp"
    convert_xyz2inp(
        osp.join(xyz_dname, xyz_fname),
        osp.join(inp_dname, inp_fname),
        cal_cmd=CAL_CMD, nproc=None, test=False)


100%|█████████████████████████████████████| 7165/7165 [00:01<00:00, 5471.35it/s]


## split inp files

In [44]:
import os
import re
import os.path as osp
from typing import Union, List, Dict
import random

from tqdm import tqdm


os.chdir(PROJECT_ROOT)
os.getcwd()

'/home/louzekun/orca_cal/qm7_ccsd_t_new'

In [48]:
inp_fnames = [i for i in os.listdir(inp_dname) if i.endswith(".inp")]
task_names = [i.replace(".inp", "") for i in inp_fnames]
print(f"{len(task_names)=}")

random.seed(0)  # set seed
random.shuffle(inp_fnames)


len(task_names)=7165


In [57]:
# split the list into N parts
def split_list(lst:List, n:int) -> List[List]:
    return [lst[i::n] for i in range(n)]

inp_fnames_split = split_list(inp_fnames, SPLIT_CNT)
task_names_split = [[j.replace(".inp", "") for j in i] for i in inp_fnames_split]

assert len(task_names_split) == SPLIT_CNT
assert sum([len(i) for i in task_names_split]) == len(inp_fnames)

print("split status: ", end="")
print(" + ".join([f"{len(i)}" for i in task_names_split]), end="")
print(f" = {len(inp_fnames)}")

split status: 2389 + 2388 + 2388 = 7165


In [60]:
# write task names of each thread to thread files
for thread_id, task_names in enumerate(task_names_split):
    with open(osp.join(threads_dname, f"thread_{thread_id}"), "w") as f:
        f.write("\n".join(task_names))